In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### EDA

https://public.tableau.com/shared/6GC9KZ6DS?:display_count=n&:origin=viz_share_link


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
train_peptides = pd.read_csv('/content/drive/Shareddrives/ML@SJSU Members/Hands-on 2023/Parkinsons Disease/Data/train_peptides.csv')
train_peptides.shape

(981834, 6)

### How many unique patients we have in the dataset?


In [ ]:
unique_patients = len(train_peptides.patient_id.unique())
print('Number of Unique Patients: ', unique_patients)

Number of Unique Patients:  248


### *Visualization* : Patient id in x axis and number of records in y axis

In [ ]:
# train_peptides.groupby('patient_id').nunique()

# patient_id, n_records = np.unique(train_peptides['patient_id'], return_counts=True)
# patient_id, n_records

# fig, ax = plt.subplots(figsize=(30,10))

# plt.bar(patient_id.astype(str), n_records)

# plt.xlabel('Patient ID')
# plt.ylabel('Number of Records')

# plt.show()

# num_of_visits = train_peptides['patient_id'].value_counts().head(10)
# num_of_visits.plot(kind='bar')

## Protein

In [ ]:
train_proteins = pd.read_csv('/content/drive/Shareddrives/ML@SJSU Members/Hands-on 2023/Parkinsons Disease/Data/train_proteins.csv')
train_proteins.shape

(232741, 5)

In [ ]:
train_proteins

,visit_id,visit_month,patient_id,UniProt,NPX
0,55_0,0,55,O00391,11254.3
1,55_0,0,55,O00533,732430.0
2,55_0,0,55,O00584,39585.8
3,55_0,0,55,O14498,41526.9
4,55_0,0,55,O14773,31238.0
...,...,...,...,...,...
232736,58648_108,108,58648,Q9UBX5,27387.8
232737,58648_108,108,58648,Q9UHG2,369437.0
232738,58648_108,108,58648,Q9UKV8,105830.0
232739,58648_108,108,58648,Q9Y646,21257.6


In [ ]:
unique_protein = np.unique(train_proteins['UniProt'])
unique_protein

### **Visualization**: Patient id in x axis and number of proteins for each patient in y axis

In [ ]:
# uniprot_per_patient = train_proteins.groupby('patient_id')['UniProt'].nunique()
# uniprot_per_patient.plot(kind='bar')

# patient_id, n_records = np.unique(train_proteins['patient_id'], return_counts=True)
# patient_id, n_records

In [ ]:
train_clinical_data = pd.read_excel('/content/drive/Shareddrives/ML@SJSU Members/Hands-on 2023/Parkinsons Disease/Data/train_clinical_data.xlsx')
train_clinical_data.shape

(2615, 8)

In [ ]:
# len(np.unique(train_clinical_data['patient_id']))

### Pivot Protein dataframe and merge with clinical data

In [ ]:
pivoted = train_proteins.pivot(index=['patient_id', 'visit_month'], columns='UniProt', values='NPX')
pivoted = pivoted.reset_index()

In [ ]:
merged = pd.merge(train_clinical_data, pivoted, on=['patient_id', 'visit_month'])
merged

In [ ]:
import missingno as msno

In [ ]:
msno.matrix(merged)

In [ ]:
msno.bar(merged)

In [ ]:
pd.set_option('display.max_rows', None)

missing_values = merged.isna().mean()*100
print(missing_values.sort_values(ascending=False))

In [ ]:
for i in unique_protein:
  merged[i] = merged.groupby('patient_id')[i].transform(lambda x: x.fillna(x.mean()))

In [ ]:
msno.matrix(merged)

In [ ]:
missing_values = merged.isna().mean()*100
print(missing_values.sort_values(ascending=False))

In [ ]:
for i in unique_protein:
  merged[i].fillna(merged[i].mean(), inplace=True)

In [ ]:
missing_values = merged.isna().mean()*100
print(missing_values.sort_values(ascending=False))

In [ ]:
-

**Ensemble: RandomForest, SVR, Linear:**

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
def smape(y_true, y_pred):
    smap = np.zeros(len(y_true))
    
    num = np.abs(y_true - y_pred)
    dem = ((np.abs(y_true) + np.abs(y_pred)) / 2)
    
    pos_ind = (y_true!=0)|(y_pred!=0)
    smap[pos_ind] = num[pos_ind] / dem[pos_ind]
    
    return 100 * np.mean(smap)

In [ ]:
peptides = pd.read_csv('/content/drive/Shareddrives/ML@SJSU Members/Hands-on 2023/Parkinsons Disease/Data/train_peptides.csv')
print('Peptides shape:',peptides.shape)
peptides.head()

Peptides shape: (981834, 6)


,visit_id,visit_month,patient_id,UniProt,Peptide,PeptideAbundance
0,55_0,0,55,O00391,NEQEQPLGQWHLS,11254.3
1,55_0,0,55,O00533,GNPEPTFSWTK,102060.0
2,55_0,0,55,O00533,IEIPSSVQQVPTIIK,174185.0
3,55_0,0,55,O00533,KPQSAVYSTGSNGILLC(UniMod_4)EAEGEPQPTIK,27278.9
4,55_0,0,55,O00533,SMEQNGPGLEYR,30838.7


In [ ]:
proteins = pd.read_csv('/content/drive/Shareddrives/ML@SJSU Members/Hands-on 2023/Parkinsons Disease/Data/train_proteins.csv')
print('Proteins shape:',proteins.shape)
proteins.head()

Proteins shape: (232741, 5)


,visit_id,visit_month,patient_id,UniProt,NPX
0,55_0,0,55,O00391,11254.3
1,55_0,0,55,O00533,732430.0
2,55_0,0,55,O00584,39585.8
3,55_0,0,55,O14498,41526.9
4,55_0,0,55,O14773,31238.0


In [ ]:
clinical = pd.read_excel('/content/drive/Shareddrives/ML@SJSU Members/Hands-on 2023/Parkinsons Disease/Data/train_clinical_data.xlsx')
print('Clinical Data shape:',clinical.shape)
clinical.head()

Clinical Data shape: (2615, 8)


,visit_id,patient_id,visit_month,updrs_1,updrs_2,updrs_3,updrs_4,upd23b_clinical_state_on_medication
0,55_0,55,0,10.0,6.0,15.0,NaN,NaN
1,55_3,55,3,10.0,7.0,25.0,NaN,NaN
2,55_6,55,6,8.0,10.0,34.0,NaN,NaN
3,55_9,55,9,8.0,9.0,30.0,0.0,On
4,55_12,55,12,10.0,10.0,41.0,0.0,On


In [ ]:
proteins_test = pd.read_csv('/content/drive/Shareddrives/ML@SJSU Members/Hands-on 2023/Parkinsons Disease/test_proteins.csv')
print('Proteins test shape:',proteins_test.shape)
proteins_test.head()

Proteins test shape: (453, 6)


,visit_id,visit_month,patient_id,UniProt,NPX,group_key
0,50423_0,0,50423,O00391,33127.90,0
1,50423_0,0,50423,O00533,490742.00,0
2,50423_0,0,50423,O00584,43615.30,0
3,50423_0,0,50423,O14773,16486.60,0
4,50423_0,0,50423,O14791,2882.42,0


In [ ]:
df_0 = clinical[(clinical.visit_month == 0)][['visit_id','updrs_1']]
print('Train shape:', df_0.shape)
df_0.head()

Train shape: (248, 2)


,visit_id,updrs_1
0,55_0,10.0
13,942_0,3.0
28,1517_0,11.0
38,1923_0,2.0
45,2660_0,2.0


In [ ]:
proteins_npx_ft = proteins.groupby('visit_id').agg(NPX_min=('NPX','min'), NPX_max=('NPX','max'), NPX_mean=('NPX','mean'), NPX_std=('NPX','std'))\
                .reset_index()
proteins_npx_ft.head()

,visit_id,NPX_min,NPX_max,NPX_mean,NPX_std
0,10053_0,2497.84,269126000.0,2.856580e+06,2.131630e+07
1,10053_12,5800.87,270030000.0,2.728871e+06,2.092162e+07
2,10053_18,1334.11,278835000.0,2.509967e+06,1.969453e+07
3,10138_12,2520.24,365582000.0,3.002583e+06,2.516170e+07
4,10138_24,1436.94,396894000.0,3.068891e+06,2.716806e+07


In [ ]:
df_proteins = pd.merge(proteins, df_0, on = 'visit_id', how = 'inner').reset_index()
proteins_Uniprot_updrs = df_proteins.groupby('UniProt').agg(updrs_1_sum = ('updrs_1','mean')).reset_index()
proteins_Uniprot_updrs.head()

,UniProt,updrs_1_sum
0,O00391,4.971014
1,O00533,5.319588
2,O00584,5.286458
3,O14498,5.217877
4,O14773,5.371585


In [ ]:
df_proteins = pd.merge(proteins, proteins_Uniprot_updrs, on = 'UniProt', how = 'left')
proteins_UniProt_ft = df_proteins.groupby('visit_id').agg(proteins_updrs_1_min=('updrs_1_sum','min'), proteins_updrs_1_max=('updrs_1_sum','max'),\
                                                          proteins_updrs_1_mean=('updrs_1_sum','mean'), proteins_updrs_1_std=('updrs_1_sum','std'))\
                .reset_index()
proteins_UniProt_ft.head()

,visit_id,proteins_updrs_1_min,proteins_updrs_1_max,proteins_updrs_1_mean,proteins_updrs_1_std
0,10053_0,4.892857,5.601449,5.300548,0.077355
1,10053_12,4.816794,5.652174,5.296073,0.099055
2,10053_18,4.297619,5.652174,5.272617,0.134631
3,10138_12,4.297619,5.652174,5.263118,0.143238
4,10138_24,4.297619,5.652174,5.269522,0.137776


In [ ]:
peptides.head()

,visit_id,visit_month,patient_id,UniProt,Peptide,PeptideAbundance
0,55_0,0,55,O00391,NEQEQPLGQWHLS,11254.3
1,55_0,0,55,O00533,GNPEPTFSWTK,102060.0
2,55_0,0,55,O00533,IEIPSSVQQVPTIIK,174185.0
3,55_0,0,55,O00533,KPQSAVYSTGSNGILLC(UniMod_4)EAEGEPQPTIK,27278.9
4,55_0,0,55,O00533,SMEQNGPGLEYR,30838.7


In [ ]:
peptides_PeptideAbundance_ft = peptides.groupby('visit_id').agg(Abe_min=('PeptideAbundance','min'), Abe_max=('PeptideAbundance','max'),\
                                                                Abe_mean=('PeptideAbundance','mean'), Abe_std=('PeptideAbundance','std'))\
                .reset_index()
peptides_PeptideAbundance_ft.head()

,visit_id,Abe_min,Abe_max,Abe_mean,Abe_std
0,10053_0,82.9679,66333900.0,726248.393431,3.535602e+06
1,10053_12,128.4460,73059300.0,737183.385744,3.799654e+06
2,10053_18,108.5000,64711200.0,601466.784320,3.006568e+06
3,10138_12,129.0240,71652400.0,699099.199189,3.379573e+06
4,10138_24,142.6480,123897000.0,732120.888877,4.912602e+06


In [ ]:
df_peptides = pd.merge(peptides, df_0, on = 'visit_id', how = 'inner').reset_index()
peptides_PeptideAbundance_updrs = df_peptides.groupby('Peptide').agg(updrs_1_sum = ('updrs_1','mean')).reset_index()
peptides_PeptideAbundance_updrs.head()

,Peptide,updrs_1_sum
0,AADDTWEPFASGK,5.357143
1,AAFGQGSGPIMLDEVQC(UniMod_4)TGTEASLADC(UniMod_4)K,5.296703
2,AAFTEC(UniMod_4)C(UniMod_4)QAADK,5.305699
3,AANEVSSADVK,5.364780
4,AATGEC(UniMod_4)TATVGKR,5.146497


In [ ]:
df_peptides = pd.merge(peptides, peptides_PeptideAbundance_updrs, on = 'Peptide', how = 'left')
peptides_ft = df_peptides.groupby('visit_id').agg(peptides_updrs_1_min=('updrs_1_sum','min'), peptides_updrs_1_max=('updrs_1_sum','max'),\
                                                          peptides_updrs_1_mean=('updrs_1_sum','mean'), peptides_updrs_1_std=('updrs_1_sum','std'))\
                .reset_index()
peptides_ft

,visit_id,peptides_updrs_1_min,peptides_updrs_1_max,peptides_updrs_1_mean,peptides_updrs_1_std
0,10053_0,4.878788,5.661972,5.279278,0.092880
1,10053_12,4.816794,5.661972,5.277513,0.097712
2,10053_18,4.297619,5.661972,5.265384,0.116303
3,10138_12,4.297619,5.661972,5.253513,0.126117
4,10138_24,4.297619,5.661972,5.257710,0.123452
...,...,...,...,...,...
1108,8699_24,4.572519,5.661972,5.256902,0.123395
1109,942_12,4.572519,5.661972,5.254323,0.118205
1110,942_24,4.572519,5.652174,5.255565,0.117226
1111,942_48,4.572519,5.652174,5.253489,0.119653


In [ ]:
df_0_1 = clinical[(clinical.visit_month == 3)][['visit_id','updrs_1']]
df_0_2 = clinical[(clinical.visit_month == 3)][['visit_id','updrs_2']]
df_0_3 = clinical[(clinical.visit_month == 3)][['visit_id','updrs_3']]
df_0_4 = clinical[(clinical.visit_month == 3)][['visit_id','updrs_4']]

df_proteins = pd.merge(proteins, df_0_1, on = 'visit_id', how = 'inner').reset_index()
proteins_Uniprot_updrs1 = df_proteins.groupby('UniProt').agg(updrs_1_sum = ('updrs_1','mean')).reset_index()

df_proteins = pd.merge(proteins, df_0_2, on = 'visit_id', how = 'inner').reset_index()
proteins_Uniprot_updrs2 = df_proteins.groupby('UniProt').agg(updrs_1_sum = ('updrs_2','mean')).reset_index()

df_proteins = pd.merge(proteins, df_0_3, on = 'visit_id', how = 'inner').reset_index()
proteins_Uniprot_updrs3 = df_proteins.groupby('UniProt').agg(updrs_1_sum = ('updrs_3','mean')).reset_index()

df_proteins = pd.merge(proteins, df_0_4, on = 'visit_id', how = 'inner').reset_index()
proteins_Uniprot_updrs4 = df_proteins.groupby('UniProt').agg(updrs_1_sum = ('updrs_4','mean')).reset_index()

df_peptides = pd.merge(peptides, df_0_1, on = 'visit_id', how = 'inner').reset_index()
peptides_PeptideAbundance_updrs1 = df_peptides.groupby('Peptide').agg(updrs_1_sum = ('updrs_1','mean')).reset_index()

df_peptides = pd.merge(peptides, df_0_2, on = 'visit_id', how = 'inner').reset_index()
peptides_PeptideAbundance_updrs2 = df_peptides.groupby('Peptide').agg(updrs_1_sum = ('updrs_2','mean')).reset_index()

df_peptides = pd.merge(peptides, df_0_3, on = 'visit_id', how = 'inner').reset_index()
peptides_PeptideAbundance_updrs3 = df_peptides.groupby('Peptide').agg(updrs_1_sum = ('updrs_3','mean')).reset_index()

df_peptides = pd.merge(peptides, df_0_4, on = 'visit_id', how = 'inner').reset_index()
peptides_PeptideAbundance_updrs4 = df_peptides.groupby('Peptide').agg(updrs_1_sum = ('updrs_4','mean')).reset_index()

df_proteins_fts = [proteins_Uniprot_updrs1, proteins_Uniprot_updrs2, proteins_Uniprot_updrs3, proteins_Uniprot_updrs4]
df_peptides_fts = [peptides_PeptideAbundance_updrs1, peptides_PeptideAbundance_updrs2, peptides_PeptideAbundance_updrs3, peptides_PeptideAbundance_updrs4]
df_lst = [df_0_1, df_0_2, df_0_3, df_0_4]

In [ ]:
def features(df, proteins, peptides, classes):
    proteins_npx_ft = proteins.groupby('visit_id').agg(NPX_min=('NPX','min'), NPX_max=('NPX','max'), NPX_mean=('NPX','mean'), NPX_std=('NPX','std'))\
                    .reset_index()
    peptides_PeptideAbundance_ft = peptides.groupby('visit_id').agg(Abe_min=('PeptideAbundance','min'), Abe_max=('PeptideAbundance','max'),\
                                                                    Abe_mean=('PeptideAbundance','mean'), Abe_std=('PeptideAbundance','std'))\
                    .reset_index()

    df_proteins = pd.merge(proteins, df_proteins_fts[classes], on = 'UniProt', how = 'left')
    proteins_UniProt_ft = df_proteins.groupby('visit_id').agg(proteins_updrs_1_min=('updrs_1_sum','min'), proteins_updrs_1_max=('updrs_1_sum','max'),\
                                                              proteins_updrs_1_mean=('updrs_1_sum','mean'), proteins_updrs_1_std=('updrs_1_sum','std'))\
                    .reset_index()
    df_peptides = pd.merge(peptides, df_peptides_fts[classes], on = 'Peptide', how = 'left')
    peptides_ft = df_peptides.groupby('visit_id').agg(peptides_updrs_1_min=('updrs_1_sum','min'), peptides_updrs_1_max=('updrs_1_sum','max'),\
                                                              peptides_updrs_1_mean=('updrs_1_sum','mean'), peptides_updrs_1_std=('updrs_1_sum','std'))\
                    .reset_index()

    df = pd.merge(df, proteins_npx_ft, on = 'visit_id', how = 'left')
    df = pd.merge(df, peptides_PeptideAbundance_ft, on = 'visit_id', how = 'left')
    df = pd.merge(df, proteins_UniProt_ft, on = 'visit_id', how = 'left')
    df = pd.merge(df, peptides_ft, on = 'visit_id', how = 'left')
    df = df.fillna(df.mean())
    return df

In [ ]:
train_0 = features(df_0_1, proteins, peptides, 0)
train_0

,visit_id,updrs_1,NPX_min,NPX_max,NPX_mean,NPX_std,Abe_min,Abe_max,Abe_mean,Abe_std,proteins_updrs_1_min,proteins_updrs_1_max,proteins_updrs_1_mean,proteins_updrs_1_std,peptides_updrs_1_min,peptides_updrs_1_max,peptides_updrs_1_mean,peptides_updrs_1_std
0,55_3,10.0,507.771,201446000.0,2.123204e+06,1.448782e+07,75.9736,59282500.0,500980.873898,2.490907e+06,6.0,6.0,6.0,0.0,6.0,6.0,6.0,0.0
1,942_3,7.0,507.771,201446000.0,2.123204e+06,1.448782e+07,75.9736,59282500.0,500980.873898,2.490907e+06,6.0,6.0,6.0,0.0,6.0,6.0,6.0,0.0
2,3636_3,4.0,507.771,201446000.0,2.123204e+06,1.448782e+07,75.9736,59282500.0,500980.873898,2.490907e+06,6.0,6.0,6.0,0.0,6.0,6.0,6.0,0.0
3,4161_3,1.0,507.771,201446000.0,2.123204e+06,1.448782e+07,75.9736,59282500.0,500980.873898,2.490907e+06,6.0,6.0,6.0,0.0,6.0,6.0,6.0,0.0
4,5645_3,5.0,507.771,201446000.0,2.123204e+06,1.448782e+07,75.9736,59282500.0,500980.873898,2.490907e+06,6.0,6.0,6.0,0.0,6.0,6.0,6.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110,62723_3,7.0,507.771,201446000.0,2.123204e+06,1.448782e+07,75.9736,59282500.0,500980.873898,2.490907e+06,6.0,6.0,6.0,0.0,6.0,6.0,6.0,0.0
111,62792_3,0.0,507.771,201446000.0,2.123204e+06,1.448782e+07,75.9736,59282500.0,500980.873898,2.490907e+06,6.0,6.0,6.0,0.0,6.0,6.0,6.0,0.0
112,64669_3,15.0,507.771,201446000.0,2.123204e+06,1.448782e+07,75.9736,59282500.0,500980.873898,2.490907e+06,6.0,6.0,6.0,0.0,6.0,6.0,6.0,0.0
113,64674_3,5.0,507.771,201446000.0,2.123204e+06,1.448782e+07,75.9736,59282500.0,500980.873898,2.490907e+06,6.0,6.0,6.0,0.0,6.0,6.0,6.0,0.0


In [6]:
import pandas as pd

from sklearn.ensemble import RandomForestRegressor

from sklearn.tree import DecisionTreeRegressor

from sklearn.linear_model import Lasso

from sklearn.model_selection import train_test_split

from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

from sklearn import metrics
from sklearn.metrics import r2_score, mean_squared_error, make_scorer

from sklearn.metrics import confusion_matrix,precision_score,recall_score,classification_report, accuracy_score
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVR  

import warnings

warnings.filterwarnings('ignore')

In [ ]:
model = {}
mms = MinMaxScaler()
n_estimators = list(range(5,100)) # number of trees in the random forest
max_features = ['auto', 'sqrt'] # number of features in consideration at every split
max_depth = [int(x) for x in np.linspace(10, 120, num = 12)] # maximum number of levels allowed in each decision tree
min_samples_split = list(range(2,11)) # minimum sample number to split a node
min_samples_leaf = list(range(1,10)) # minimum sample number that can be stored in a leaf node
bootstrap = [True, False] # method used to sample data points


for i in range(3):
    print('--------------------------------------------------------')
    print('Model {0}'.format(i + 1))
    train_0 = features(df_lst[i], proteins, peptides, i)
    scale_col = ['NPX_min','NPX_max','NPX_mean','NPX_std', 'Abe_min', 'Abe_max', 'Abe_mean', 'Abe_std']
    train_0[scale_col] = mms.fit_transform(train_0[scale_col])
    
    rfc = RandomForestRegressor()
    sv = SVR()
    ln = Lasso()
    dt = DecisionTreeRegressor()
    
    svc_params = [{'kernel':['poly','rbf','sigmoid','linear']}]
    forest_params = [{'n_estimators': n_estimators,

    'max_features': max_features,

    'max_depth': max_depth,

    'min_samples_split': min_samples_split,

    'min_samples_leaf': min_samples_leaf,

        'bootstrap': bootstrap}]
    linear_params = [{'alpha':[0.01,0.02,0.03,0.04,0.05,0.06,0.07,0.08,0.09,0.1]}]
    tree_params = [{

    'max_features': max_features,

    'max_depth': max_depth,

    'min_samples_split': min_samples_split,

    'min_samples_leaf': min_samples_leaf}]
    
    cv = KFold(n_splits=10, shuffle=True, random_state=42)
    clf = RandomizedSearchCV(rfc, forest_params, cv = cv, scoring=make_scorer(smape), verbose = -1)
    svc = RandomizedSearchCV(sv, svc_params, cv = cv, scoring=make_scorer(smape), verbose = -1)
    linear = RandomizedSearchCV(ln, linear_params, cv = cv, scoring=make_scorer(smape), verbose = -1)
    dec = RandomizedSearchCV(dt, tree_params, cv = cv, scoring=make_scorer(smape), verbose = -1)
    
    X = train_0.drop(columns = ['visit_id','updrs_{0}'.format(i + 1)], axis = 1)
    y = train_0['updrs_{0}'.format(i + 1)].astype(np.float32)
    clf.fit(X, y)
    svc.fit(X,y)
    linear.fit(X,y)
    dec.fit(X,y)

    print('rfc best params:',clf.best_params_)

    print('rfc best score',clf.best_score_)
    
    print('svc best param',svc.best_params_)

    print('svc best score',svc.best_score_)
    
    print('linear best param',linear.best_params_)

    print('linear best score',linear.best_score_)
    
    print('decision tree best param',dec.best_params_)

    print('decision tree best score',dec.best_score_)
    
    print('Train smape rfc:',smape(train_0['updrs_{0}'.format(i + 1)], clf.predict(train_0.drop(columns = ['visit_id','updrs_{0}'.format(i + 1)], axis = 1))))
    print('Train smape svc:',smape(train_0['updrs_{0}'.format(i + 1)], svc.predict(train_0.drop(columns = ['visit_id','updrs_{0}'.format(i + 1)], axis = 1))))
    print('Train smape linear:',smape(train_0['updrs_{0}'.format(i + 1)], linear.predict(train_0.drop(columns = ['visit_id','updrs_{0}'.format(i + 1)], axis = 1))))
    print('Train smape decision tree:',smape(train_0['updrs_{0}'.format(i + 1)], dec.predict(train_0.drop(columns = ['visit_id','updrs_{0}'.format(i + 1)], axis = 1))))
    
    
    model['rfc_' + str(i)] = clf
    model['svc_' + str(i)] = svc
    model['linear_' + str(i)] = linear
    model['dec_' + str(i)] = dec

--------------------------------------------------------
Model 1
rfc best params: {'n_estimators': 93, 'min_samples_split': 2, 'min_samples_leaf': 6, 'max_features': 'sqrt', 'max_depth': 30, 'bootstrap': True}
rfc best score 71.49260320918508
svc best param {'kernel': 'rbf'}
svc best score 71.75552241110057
linear best param {'alpha': 0.01}
linear best score 71.43109434656283
decision tree best param {'min_samples_split': 8, 'min_samples_leaf': 9, 'max_features': 'auto', 'max_depth': 50}
decision tree best score 71.43109434656283
Train smape rfc: 70.53719875985819
Train smape svc: 70.49687673914481
Train smape linear: 70.46046849710056
Train smape decision tree: 70.46046849710056
--------------------------------------------------------
Model 2
rfc best params: {'n_estimators': 35, 'min_samples_split': 4, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'max_depth': 80, 'bootstrap': True}
rfc best score 65.62976548389551
svc best param {'kernel': 'poly'}
svc best score 63.45952754747729
l

**DL:**

In [20]:
import sys
sys.path.append('/kaggle/input/amp-pd')

In [21]:
import amp_pd_peptide
from sklearn.linear_model import LinearRegression
import pandas as pd
import numpy as np

ModuleNotFoundError: ignored

In [4]:
df = pd.read_csv('/content/drive/Shareddrives/ML@SJSU Members/Hands-on 2023/Parkinsons Disease/Data/cleaned_data.csv')

In [11]:
model = {}
target = ["updrs_1", "updrs_2", "updrs_3", "updrs_4"]

for u in target:
    temp = df.dropna(subset=[u]) 

    X = temp['visit_month']
    y = temp[u]
        
    trained = LinearRegression().fit(X.values.reshape(-1, 1), y)
    model[u] = trained

In [12]:
def get_predictions(my_train, pro, model):
    my_train = my_train.fillna(0)
    
    for u in target:
        my_train['result_' + str(u)] = 0
   
        X = my_train["visit_month"]
        
        if u == 'updrs_4':
            my_train['result_' + str(u)] = 0
        else:
            my_train['result_' + str(u)] = np.ceil(model[u].predict(X.values.reshape(-1, 1)))

    result = pd.DataFrame()

    for m in [0, 6, 12, 24]:
        for u in [1, 2, 3, 4]:
            temp = my_train[["visit_id", "result_updrs_" + str(u)]]
            temp["prediction_id"] = temp["visit_id"] + "_updrs_" + str(u) + "_plus_" + str(m) + "_months"
            temp["rating"] = temp["result_updrs_" + str(u)]
            temp = temp [['prediction_id', 'rating']]
            result = result.append(temp)            
    result = result.drop_duplicates(subset=['prediction_id', 'rating'])

    return result

In [15]:
env = amp_pd_peptide.make_env()   # initialize the environment
iter_test = env.iter_test()

NameError: ignored